In [149]:
#!wget https://raw.githubusercontent.com/karpathy/char_rnn/master/tiny/shakespeare/input.txt

In [150]:
with open('input.txt','r',encoding='utf-8') as f:
  text = f.read()

In [151]:
# 자연어 간단 처리과정
# content minibatch size로 토큰화 -> 임베딩 벡터로 임베딩 -> neural net -> backprop

In [152]:
len(text)
# text[:1000]

1115394

In [153]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars)) # 셰익스피어 txt 모든 문자


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [154]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s : [stoi[c] for c in s]
# s = string c = char 문자열마다 숫자로 바꿈
# decode = lambda i : [itos[n] for n in i]
# decode= lambda l : ''.join([itos[n] for n in l])
decode = lambda l: ''.join([itos[i] for i in l])
print(encode("hii there"))
print(decode(encode('hello world')))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hello world


In [155]:
vocab_size


65

In [156]:
import torch
data = torch.tensor(encode(text),dtype = torch.long) #매우간단한 수동 인코딩
# openai chatgpt 는 tiktoken 이란 bpe tokenizer 인코딩 방식사용
# google 은 sentencepiece라는 걸사용
data.type
print(data.shape)
print(data[:100])

torch.Size([1115394])
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [157]:
n = int(len(text)*0.9)
train = data[:n]
val = data[n:]
blocksize = 8 #WAVENET
train[:blocksize+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [158]:
x = train[:blocksize]
y= train[1:blocksize+1]
for t in range(blocksize):
  context = x[:t+1]
  target = y[t]

# x=target string 전까지의 모든 string의 tokenized 된int, y= target


  print(f'입력값이 {context} 일 때 입력값 sequence의 다음값은 {target}')

입력값이 tensor([18]) 일 때 입력값 sequence의 다음값은 47
입력값이 tensor([18, 47]) 일 때 입력값 sequence의 다음값은 56
입력값이 tensor([18, 47, 56]) 일 때 입력값 sequence의 다음값은 57
입력값이 tensor([18, 47, 56, 57]) 일 때 입력값 sequence의 다음값은 58
입력값이 tensor([18, 47, 56, 57, 58]) 일 때 입력값 sequence의 다음값은 1
입력값이 tensor([18, 47, 56, 57, 58,  1]) 일 때 입력값 sequence의 다음값은 15
입력값이 tensor([18, 47, 56, 57, 58,  1, 15]) 일 때 입력값 sequence의 다음값은 47
입력값이 tensor([18, 47, 56, 57, 58,  1, 15, 47]) 일 때 입력값 sequence의 다음값은 58


In [159]:
torch.manual_seed(1337)

batchsize = 4 #병렬처리
blocksize = 8 #예측에 사용되는 최대 context길이

def get_batch(split):
  data = train if split == 'train' else val
  ix =torch.randint(len(data)-blocksize,(batchsize,)) #0~len(data)-blocksize에서 랜덤한 정수
  # 총 데이터에서 4개씩 idx설정
  x = torch.stack([data[i:i+blocksize] for i in ix])  #chunking
  # 위의 짧은 snippet과 동일한과정 but 전체 data에 for문을 처리해 적용
  y = torch.stack([data[i+1:i+blocksize+1] for i in ix])
  return x,y

  # ix shape (4,) data shape (1115394) data[ix:ix+blocksize] shape (4,8)

xb, yb = get_batch('train')
print(xb,xb.shape)
print('-------------------------------------------------')
print(yb.shape,yb)
for b in range(batchsize):  #batch dimension
  for t in range(blocksize):  #time dimension  e.g wavenet B,T,C
    context  = xb[b,:t+1]
    target = yb[b,t]
    print(f'입력값이 {context.tolist()} 일때 입력값 sequence 다음값은 {target}')

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) torch.Size([4, 8])
-------------------------------------------------
torch.Size([4, 8]) tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
입력값이 [24] 일때 입력값 sequence 다음값은 43
입력값이 [24, 43] 일때 입력값 sequence 다음값은 58
입력값이 [24, 43, 58] 일때 입력값 sequence 다음값은 5
입력값이 [24, 43, 58, 5] 일때 입력값 sequence 다음값은 57
입력값이 [24, 43, 58, 5, 57] 일때 입력값 sequence 다음값은 1
입력값이 [24, 43, 58, 5, 57, 1] 일때 입력값 sequence 다음값은 46
입력값이 [24, 43, 58, 5, 57, 1, 46] 일때 입력값 sequence 다음값은 43
입력값이 [24, 43, 58, 5, 57, 1, 46, 43] 일때 입력값 sequence 다음값은 39
입력값이 [44] 일때 입력값 sequence 다음값은 53
입력값이 [44, 53] 일때 입력값 sequence 다음값은 56
입력값이 [44, 53, 56] 일때 입력값 sequence 다음값은 1
입력값이 [44, 53, 56, 1] 일때 입력값 sequence 다음값은 58
입력값이 [44, 53, 56, 1, 58] 일때 입력값 sequence 다

In [160]:
from torch.nn import functional as F
import torch.nn as nn

In [163]:
torch.manual_seed(1337)
class blm(nn.Module): #bigram language model
  def __init__(self,vocab_size):
      super().__init__()
      self.token_emb_table = nn.Embedding(vocab_size,vocab_size)

  def forward(self,idx, targets=None):
      # idx,xb =4,8 idx의미 embedding matrix의 idx = 단어정보의 int를 batchsize로 묶은 벡터
      # idx,targets 모두 B,T shape tensor
      logits = self.token_emb_table(idx)  #C[xb]와 유사 wavenet

      if targets is None:
        loss = None
      else:
        B,T,C = logits.shape
        logits = logits.view(B*T,C) #logits shpae =B,T,C #벡터값으로 reshape
        targets = targets.view(B*T) #스칼라값
        loss = F.cross_entropy(logits,targets)

      return logits,loss



# 이전에 생성된 토큰들에 대한 정보를 유지하면서 새로운 토큰을 생성함.
# 이는 텍스트 생성과 같은 작업에서 이전 문맥을 유지하면서 연속된 문장이나
# 시퀀스를 생성하는 데 사용됨. 밑의 idx generate과정이 이에 해당함
  def generate(self,idx,max_new_tokens):
    #idx shape = B,T
      for _ in range(max_new_tokens):
        #예측값을 얻음 loss는 계산되지않음
        # 위의 target이 optional 이기 때문에 argument idx 한개여도 forward 실행가능
        # self 를 instance로
        logits, loss = self(idx)
        logits = logits[:,-1,:] #embedding된 토큰에서의 timestep의 마지막 토큰
        probs = F.softmax(logits,dim=-1)
        idx_next = torch.multinomial(probs,num_samples=1)
        idx = torch.cat((idx,idx_next),dim=1)
      return idx

m= blm(vocab_size)
logits,loss = m(xb,yb) #forward pass 인스턴스를 함수로써 __call__과 유사
print(logits.shape) #65 = vocab size
print(loss)
# -ln(1/65)와 loss값이 다름으로 prediciton 잘못된 generate로 idx형성
# idx = torch.zeros((1,1),dtype= torch.long)
print(decode(m.generate(idx = torch.zeros((1,1),dtype= torch.long),max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [166]:
#forward pass done
#훈련 ㄱ
optimizer = torch.optim.AdamW(m.parameters(),lr = 1e-3)
batchsize =32
for steps in range(10000):
  # xb = idx, yb = targets
  xb,yb = get_batch('train')

  logits,loss = m(xb,yb)
  # 항상하는 gradient 초기값설정 과정
  # for p in parameters: p.grad =None parameters = [p for layer in layers for p in layer.parameter()]
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())


2.498678684234619


torch.Size([4, 8, 2])

In [175]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x= torch.randn(B,T,C)
x.shape
xbow = torch.zeros((B,T,C)) #x의 평균 저장할 공간
# bow = bag of words 평균낼때 사용
for b in range(B):
  for t in range(T):
    # xprev shape (t,C)
    # batch dimentsion는 정해진값, t를 포함한 t까지의 time token까지 slicing
    xprev = x[b,:t+1]
    # torch.mean(xprev,0) shape 는 xprev의 0:t에서의 평균을 구함
    xbow[b,t] = torch.mean(xprev,0) #row에서 xprev[0]에서 mean

# 밑의 x표, xprev 는 x=[[ 0.1808, -0.0700],[-0.3596, -0.9152],...]에서
  # t가0일때는 # [ 0.1808, -0.0700] 0.1808 의평균
  # t가1일떄는 [ 0.1808, -0.0700],[-0.3596, -0.9152]의 row값에서의 평균
  # t가 2일떄 0.1808 -0.3596 0.6258 평균 row에서의 평균
  #이값을 xbow에 채워 넣음

#       ([[[ 0.1808, -0.0700],
        #  [-0.3596, -0.9152],
        #  [ 0.6258,  0.0255],
        #  [ 0.9545,  0.0643],
        #  [ 0.3612,  1.1679],
        #  [-1.3499, -0.5102],
        #  [ 0.2360, -0.2398],
        #  [-0.9211,  1.5433]],

        # [[ 1.3488, -0.1396],
        #  [ 0.2858,  0.9651],
        #  [-2.0371,  0.4931],
        #  [ 1.4870,  0.5910],
        #  [ 0.1260, -1.5627],
        #  [-1.1601, -0.3348],
        #  [ 0.4478, -0.8016],
        #  [ 1.5236,  2.5086]],
print(x,xbow)

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]],

        [[ 1.3488, -0.1396],
         [ 0.2858,  0.9651],
         [-2.0371,  0.4931],
         [ 1.4870,  0.5910],
         [ 0.1260, -1.5627],
         [-1.1601, -0.3348],
         [ 0.4478, -0.8016],
         [ 1.5236,  2.5086]],

        [[-0.6631, -0.2513],
         [ 1.0101,  0.1215],
         [ 0.1584,  1.1340],
         [-1.1539, -0.2984],
         [-0.5075, -0.9239],
         [ 0.5467, -1.4948],
         [-1.2057,  0.5718],
         [-0.5974, -0.6937]],

        [[ 1.6455, -0.8030],
         [ 1.3514, -0.2759],
         [-1.5108,  2.1048],
         [ 2.7630, -1.7465],
         [ 1.4516, -1.5103],
         [ 0.8212, -0.2115],
         [ 0.7789,  1.5333],
         [ 1.6097, -0.4032]]]) tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
      

In [ ]:
# 매우 복잡함으로 mathmetical trick comes in


In [ ]:

#